# Data cleansing notes on removed records:

## See data_engineering.ipynb for more details.

### Measurement Records Information
    19,550 total records before changes
    1,447 records with blank precipitation (col = "prcp")
    18,103 records with no known issues after removing 1,447 with blank prcp
    Percentage of records removed is 0.07%
    Percentage of records remaining is 93.0%


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy import func

In [ ]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")
# Create a connection to the engine called `conn`
conn = engine.connect()

In [ ]:
# Use `declarative_base` from SQLAlchemy to model the weather table as an ORM class
Base = declarative_base()

class weather(Base):
    __tablename__ = 'weather'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
def __repr__(self):
    return "<Weather(%r, %r, %r, %r, %r)>" % (self.id, self.station, self.date, 
                                              self.prcp, self.tobs)

In [ ]:
# Display the schema of the table we just created
weather.__table__

In [ ]:
# Use `create_all` to create the weather table
Base.metadata.create_all(engine)

In [ ]:
# Load the cleaned csv file into a pandas dataframe
df_w_data_to_insert = pd.read_csv("clean_weather_source.csv")

In [ ]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
w_data = df_w_data_to_insert.to_dict(orient='records')
w_data[0]

In [ ]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
# Save the reference to the `weather` table as a variable called `table`
w_table = sqlalchemy.Table('weather', metadata, autoload=True)

In [ ]:
# Use `declarative_base` from SQLAlchemy to model the weather table as an ORM class
Base = declarative_base()

class station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station   = Column(Text)
    name      = Column(Text)
    latitude  = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
def __repr__(self):
    return "<Station(%r, %r, %r, %r, %r, %r)>" % (self.id, self.station, self.name, 
                                                  self.latitude, self.longitude,
                                                  self.elevation)

In [ ]:
# Display the schema of the table we just created
station.__table__

In [ ]:
# Use `create_all` to create the station table
Base.metadata.create_all(engine)

In [ ]:
# Load the cleaned csv file into a pandas dataframe
df_s_data_to_insert = pd.read_csv("clean_station_source.csv")

In [ ]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
s_data = df_s_data_to_insert.to_dict(orient='records')
s_data[0]

In [ ]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
# Save the reference to the `station` table as a variable called `table`
s_table = sqlalchemy.Table('station', metadata, autoload=True)

In [ ]:
# Use _table.insert() to insert the data into the table
conn.execute(w_table.insert(), w_data)

In [ ]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from weather limit 5").fetchall()

In [ ]:
# Use _table.insert() to insert the data into the table
conn.execute(s_table.insert(), s_data)

In [ ]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

In [ ]:
# list tables in database hawaii
tables = engine.execute("SELECT * FROM sqlite_master WHERE TYPE='table'")
for index, table in enumerate(tables):
    print(table)
    print()
    table_name = table[1]
    print(f"The #{index} table in DB hawaii is called '{table_name}'.")

In [ ]:
# https://sqlite.org/pragma.html
# list colums in the weather table
columns = list(engine.execute("PRAGMA table_info(weather)"))
print(columns)
print()
for column in columns:
    (number, name, type, _, _, _) = column
    print(f"Column {number} is called '{name}', and is of type '{type}'.")

In [ ]:
# https://sqlite.org/pragma.html
# list the columns in the Station table
columns = list(engine.execute("PRAGMA table_info(station)"))
print(columns)
print()
for column in columns:
    (number, name, type, _, _, _) = column
    print(f"Column {number} is called '{name}', and is of type '{type}'.")

In [ ]:
# Create a session to run SQL
from sqlalchemy.orm import Session
from sqlalchemy import func
# Declare a session
session = Session(bind=engine)

In [ ]:
# Get an actual number for count(*) by using scalar()
session.query(func.count(station.id)).scalar()

In [ ]:
# Get an actual number for count(*) by using scalar()
session.query(func.count(weather.id)).scalar()

In [ ]:
# Use table.delete() to remove any existing data and re-run.
# conn.execute(w_table.delete())
# conn.execute(s_table.delete())